In [1]:
import os
import json

import numpy as np
import tensorflow as tf

import utils_CNN as utils

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# solve the problem of "libdevice not found at ./libdevice.10.bc"
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/home/r10222035/.conda/envs/tf2'

2024-06-04 15:30:36.649287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 15:30:36.722811: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def sculpting_sensitivity(SR_eff, SB_eff, B):
    # SR_eff: background efficiency in signal region
    # SB_eff: background efficiency in sideband region
    # B: number of background events in signal region
    
    nS = B * (SR_eff - SB_eff)
    nB = B * SB_eff
    return nS / nB**0.5

In [3]:
def get_SRfpr_from_SBfpr(X_SRSB, y_SRSB, model_name, bkg_effs=[0.1]):
    # get the fpr in signal region from the fpr in sideband region
    # fpr: false positive rate, background efficiency

    save_model_name = f'./CNN_models/last_model_CWoLa_hunting_{model_name}/'
    loaded_model = tf.keras.models.load_model(save_model_name)

    X_SR, X_SB = X_SRSB
    y_SR, y_SB = y_SRSB

    y_prob_SB = loaded_model.predict(X_SB)
    fpr_SB, th_SB = utils.get_fpr_thresholds(y_SB == 1, y_prob_SB)

    y_prob_SR = loaded_model.predict(X_SR)
    fpr_SR, th_SR = utils.get_fpr_thresholds(y_SR == 1, y_prob_SR)


    bkg_effs_SR = []
    for bkg_eff in bkg_effs:
        th_SB_bkg_eff = utils.get_threshold_from_fpr(fpr_SB, th_SB, bkg_eff)
        n_th = (th_SR < th_SB_bkg_eff).sum()
        bkg_effs_SR.append(fpr_SR[n_th])

    return bkg_effs_SR

## Original

In [4]:
config_file = 'config_files/origin_config_01.json'
with open(config_file) as f:
    config = json.load(f)

train_npy_paths = config['train_npy_paths']
seed = config['seed']
sensitivity = config['sensitivity']

model_name = 'SB_0.0_25x25'

# Training and validation splitting ratio
r_train, r_val = 0.8, 0.2

n_SR_S, n_SR_B, n_SB_S, n_SB_B = utils.compute_nevent_in_SR_SB(sensitivity=sensitivity)

train_nevents = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
X_train_SR, y_train_SR, X_train_SB, y_train_SB  = utils.get_SR_SB_sample_from_npy(train_npy_paths, train_nevents, seed=seed)

Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']


In [5]:
SB_effs = [0.1, 0.01, 0.001]
SR_effs = get_SRfpr_from_SBfpr((X_train_SR, X_train_SB), (y_train_SR, y_train_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

2024-06-04 15:30:45.406330: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 15:30:46.017232: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46699 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:3b:00.0, compute capability: 8.6
2024-06-04 15:30:48.829640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


 34/506 [=>............................] - ETA: 2s

2024-06-04 15:30:50.444810: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


474/474 [==============================] - 2s 5ms/step
 0.100  0.1238  10.4
 0.010  0.0202  14.1
 0.001  0.0022  5.4


In [6]:
X_test_SR, y_test_SR = utils.load_samples('../Sample/HVmodel/data/split_val/25x25/mix_sample_test.npy')
X_test_SB, y_test_SB = utils.load_samples('../Sample/HVmodel/data/split_val/25x25/mix_sample_test-SB.npy')

In [7]:
SR_effs = get_SRfpr_from_SBfpr((X_test_SR, X_test_SB), (y_test_SR, y_test_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

625/625 [==============================] - 3s 5ms/step
 0.100  0.1071  3.1
 0.010  0.0172  9.9
 0.001  0.0006 -1.7


## +3 Jet rotation

In [8]:
config_file = 'config_files/jet_aug_3_config_01.json'
with open(config_file) as f:
    config = json.load(f)

train_npy_paths = config['train_npy_paths']
seed = config['seed']
sensitivity = config['sensitivity']

model_name = config['model_name']

# Training and validation splitting ratio
r_train, r_val = 0.8, 0.2

n_SR_S, n_SR_B, n_SB_S, n_SB_B = utils.compute_nevent_in_SR_SB(sensitivity=sensitivity)

train_nevents = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
X_train_SR, y_train_SR, X_train_SB, y_train_SB  = utils.get_SR_SB_sample_from_npy(train_npy_paths, train_nevents, seed=seed)

Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25', '../Sample/HVmodel/data/jet_rotation/25x25/01', '../Sample/HVmodel/data/jet_rotation/25x25/02', '../Sample/HVmodel/data/jet_rotation/25x25/03']


In [9]:
SB_effs = [0.1, 0.01, 0.001]
SR_effs = get_SRfpr_from_SBfpr((X_train_SR, X_train_SB), (y_train_SR, y_train_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

1893/1893 [==============================] - 10s 5ms/step
 0.100  0.1421  18.3
 0.010  0.0268  23.1
 0.001  0.0011  0.5


In [10]:
X_test_SR, y_test_SR = utils.load_samples('../Sample/HVmodel/data/split_val/25x25/mix_sample_test.npy')
X_test_SB, y_test_SB = utils.load_samples('../Sample/HVmodel/data/split_val/25x25/mix_sample_test-SB.npy')

In [11]:
SR_effs = get_SRfpr_from_SBfpr((X_test_SR, X_test_SB), (y_test_SR, y_test_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

625/625 [==============================] - 3s 5ms/step
 0.100  0.1118  5.1
 0.010  0.0216  16.0
 0.001  0.0003 -3.0


## +3 $p_\text{T}$ smearing + Jet rotation

In [12]:
config_file = 'config_files/pt_jet_aug_3_config_01.json'
with open(config_file) as f:
    config = json.load(f)

train_npy_paths = config['train_npy_paths']
seed = config['seed']
sensitivity = config['sensitivity']

model_name = config['model_name']

# Training and validation splitting ratio
r_train, r_val = 0.8, 0.2

n_SR_S, n_SR_B, n_SB_S, n_SB_B = utils.compute_nevent_in_SR_SB(sensitivity=sensitivity)

train_nevents = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
X_train_SR, y_train_SR, X_train_SB, y_train_SB  = utils.get_SR_SB_sample_from_npy(train_npy_paths, train_nevents, seed=seed)

Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25', '../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/01', '../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/02', '../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/03']


In [13]:
SB_effs = [0.1, 0.01, 0.001]
SR_effs = get_SRfpr_from_SBfpr((X_train_SR, X_train_SB), (y_train_SR, y_train_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

1893/1893 [==============================] - 10s 5ms/step
 0.100  0.1318  13.9
 0.010  0.0212  15.4
 0.001  0.0020  4.4


In [14]:
X_test_SR, y_test_SR = utils.load_samples('../Sample/HVmodel/data/split_val/25x25/mix_sample_test.npy')
X_test_SB, y_test_SB = utils.load_samples('../Sample/HVmodel/data/split_val/25x25/mix_sample_test-SB.npy')

In [15]:
SR_effs = get_SRfpr_from_SBfpr((X_test_SR, X_test_SB), (y_test_SR, y_test_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

625/625 [==============================] - 3s 5ms/step
 0.100  0.1148  6.4
 0.010  0.0163  8.7
 0.001  0.0011  0.4
